<a href="https://colab.research.google.com/github/michelrp/IA_Gemini_Alura_Google/blob/main/SearchInProcess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando a biblioteca do Gemini

---



In [1]:
!pip install -U -q google-generativeai

# Importando bibliotecas e Setando API Key

---



In [3]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Populando base que será utilizada na busca

---

> TODO: Aqui mudar abordagem para realizar processamento dos POPs (procedimentos operacionais padrões) em PDF.

In [10]:
DOCUMENT1 = {
    "title": "Introdução ao Processo",
    "content": "Após a aceitação da oferta de emprego, o departamento de Recursos Humanos (RH) da empresa entra em contato com o novo funcionário para solicitar a entrega de documentos necessários. Este contato inicial inclui instruções detalhadas sobre quais documentos são necessários e como devem ser enviados."}

DOCUMENT2 = {
    "title": "Documentos Requeridos",
    "content": "Os documentos frequentemente solicitados incluem documento de identidade com foto, CPF, comprovante de residência, Carteira de Trabalho e Previdência Social (CTPS) digital, título de eleitor, documento de reservista para homens de 18 a 45 anos, comprovantes de qualificações e experiências anteriores, e dados bancários para depósito de salário."}

DOCUMENT3 = {
    "title": "Métodos de Envio",
    "content": "O envio dos documentos pode ser realizado de maneira digital ou física. Digitalmente, o funcionário pode escanear ou fotografar os documentos e enviá-los através de um sistema seguro de upload ou via e-mail. Fisicamente, especialmente quando necessário entregar documentos originais, o envio pode ser feito pessoalmente ou por correio."}

DOCUMENT4 = {
    "title": "Verificação e Arquivamento",
    "content": "Após o recebimento, o RH verifica a validade e a integridade dos documentos. Se houver qualquer inconformidade ou falta de algum documento, o funcionário é contatado para regularizar a situação. Após a verificação, os documentos são arquivados no prontuário do funcionário, seja digitalmente ou em papel, de acordo com as práticas da empresa."}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4]

# Criando DataFrame

---

In [15]:
df = pd.DataFrame(documents)
df.columns = ["title", "content"]
df

,title,content
0,Introdução ao Processo,"Após a aceitação da oferta de emprego, o depar..."
1,Documentos Requeridos,Os documentos frequentemente solicitados inclu...
2,Métodos de Envio,O envio dos documentos pode ser realizado de m...
3,Verificação e Arquivamento,"Após o recebimento, o RH verifica a validade e..."


# Setando modelo que será utilizado

---

In [16]:
model = "models/embedding-001"

# Criando função para calcular os Embeddins para incorporar os textos dos documentos


---

In [17]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

# Criando os Embeddings do DataFrame

---

In [18]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["title"], row["content"]), axis=1)
df

,title,content,Embeddings
0,Introdução ao Processo,"Após a aceitação da oferta de emprego, o depar...","[0.007202828, -0.012865682, -0.042857945, 0.02..."
1,Documentos Requeridos,Os documentos frequentemente solicitados inclu...,"[0.004248983, -0.0313563, -0.08839521, -0.0069..."
2,Métodos de Envio,O envio dos documentos pode ser realizado de m...,"[0.02050137, -0.031100389, -0.04411774, 0.0005..."
3,Verificação e Arquivamento,"Após o recebimento, o RH verifica a validade e...","[0.012374565, -0.024739072, -0.05896889, 0.059..."


# Criando função para gerar os embeddings da consulta do usuário sobre os embeddings do documento gerado anteriormente

---

In [19]:
def searchInProcess(query, base, model):
  embedding_query = genai.embed_content(model=model,
                                 content=query,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  scalar_products = np.dot(np.stack(df["Embeddings"]), embedding_query)

  indice = np.argmax(scalar_products)
  return df.iloc[indice]["content"]

# Prompts para testar o modelo

---

In [24]:
#question = "Quais documentos necessários?"
question = "Como funciona o processo de contratação?" #@param {type: 'string'}

answer = searchInProcess(question, df, model)
print(answer)

Após a aceitação da oferta de emprego, o departamento de Recursos Humanos (RH) da empresa entra em contato com o novo funcionário para solicitar a entrega de documentos necessários. Este contato inicial inclui instruções detalhadas sobre quais documentos são necessários e como devem ser enviados.


# Configurações do modelo

---

In [22]:
generation_config = {
  "temperature": 0,
  "candidate_count": 1
}

# Testando outro estilo de resposta

---

In [23]:
prompt = f"Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texto: {answer}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Mandar os papéis pode ser na boa e velha forma física ou no estilo digital.

Na versão digital, você pode tirar uma foto ou escanear os documentos e mandar por um site seguro ou e-mail.

Já na física, quando você precisa entregar os originais, pode ir pessoalmente ou mandar pelo correio.
